<a href="https://colab.research.google.com/github/alikaiser12/AI/blob/main/LangChain_Custom_Tool_Integration_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# ===============================================
# Single-Cell Colab Demo (Fixed for AsyncCallbackManager error):
# Uses modern package structure: langchain-core, langchain-community, and langchain
# and imports FakeListLLM from langchain_core.language_models.fake.
# ===============================================

# 1) Install compatible versions
# These versions avoid the AsyncCallbackManager import problem.
!pip install -q -U "pydantic>=2.7,<3" \
                  "langchain>=0.3.0,<0.4" \
                  "langchain-core>=0.3.0,<0.4" \
                  "langchain-community>=0.3.0,<0.4"

# 2) Imports (note the langchain_core paths)
from langchain_core.tools import StructuredTool
from langchain_core.language_models.fake import FakeListLLM
from langchain.agents import initialize_agent, AgentType

# 3) Mock API function
def query_order_status(order_id: str) -> str:
    """Return a fake order status for the given order_id."""
    return "Shipped" if order_id == "123" else "In Process"

# 4) Create the custom tool via StructuredTool.from_function
# (No args_schema needed — introspection handles the signature)
order_checker_tool = StructuredTool.from_function(
    func=query_order_status,
    name="OrderCheckerTool",
    description=(
        "Check the status of an order by ID. "
        "Use this tool when the user asks for an order status and provides the order_id."
    ),
)

# 5) Simulated LLM responses
# Each response instructs the agent to use the tool, then finalizes the answer.
fake_responses = [
    "I'll check the first order via OrderCheckerTool.\nAction: OrderCheckerTool\nAction Input: 123",
    "Got the first result. Now I'll check the second order.\nAction: OrderCheckerTool\nAction Input: 999",
    "Final Answer: Order 123 is Shipped and Order 999 is In Process."
]
llm = FakeListLLM(responses=fake_responses)

# 6) Initialize agent and run test query
agent = initialize_agent(
    tools=[order_checker_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True  # Prints thought, tool calls, observations
)

query = "I have two order IDs: '123' and '999'. What is the status of each of these orders?"
print("User Query:", query)
result = agent.run(query)

# 7) Final, client-facing confirmation
print("\n" + "="*60)
print("Final Answer:")
print(result)
print("="*60)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.5/449.5 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


TypeError: args_schema must be a subclass of pydantic BaseModel. Got: <class 'pydantic.v1.main.OrderCheckerToolSchema'>.